In [1]:
#imports
import pandas as pd
import numpy as np
import random

In [2]:
# set random seed
random.seed(42)
np.random.seed(42)

In [6]:
num_rows = 500
dates = pd.date_range(start='2023-01-01', periods=num_rows, freq='D')
regions = ["Kistarcsa", "Budapest", "Debrecen", "Szeged", "Miskolc"]
departments = ["HR", "IT", "Finance", "Marketing", "Sales"]
salaries = np.random.randint(40000, 120000, size=num_rows)

data = {
    "Date": dates,
    "Region": [random.choice(regions) for _ in range(num_rows)],
    "Department": [random.choice(departments) for _ in range(num_rows)],
    "Salary": salaries,
    "Performance_Score": np.random.randint(1, 11, size=num_rows)
}

df_company = pd.DataFrame(data)

In [13]:
for _ in range(30):
    ix = np.random.randint(0, 50)
    col = random.choices(['Department', 'Salary'], weights=[0.3,0.7])[0]
    df_company.at[ix, col] = np.nan

Nézd meg a data frame első 5 valamint utolsó 5 sorát

In [14]:
df_company.head()

,Date,Region,Department,Salary,Performance_Score
0,2023-01-01,Miskolc,HR,45249.0,8
1,2023-01-02,Debrecen,Finance,59201.0,4
2,2023-01-03,Budapest,NaN,98576.0,8
3,2023-01-04,Miskolc,Sales,NaN,7
4,2023-01-05,Szeged,Finance,NaN,5


In [15]:
df_company.tail()

,Date,Region,Department,Salary,Performance_Score
495,2024-05-10,Miskolc,IT,55892.0,6
496,2024-05-11,Budapest,Finance,87299.0,1
497,2024-05-12,Szeged,Sales,57073.0,4
498,2024-05-13,Miskolc,Sales,70682.0,1
499,2024-05-14,Szeged,HR,54289.0,2


Számold meg, hogy hány hiányzó elem van az adathalmazban

In [16]:
df_company.isna().sum()

Date                  0
Region                0
Department            8
Salary               17
Performance_Score     0
dtype: int64

Kezeld a hiányzó elemeket a következő módon: <br>
Department -> a régió legfrekventáltabb department-je <br>
Salary -> A régió és department átlaga alapján

In [17]:
# fill in the department by the most frequent department of the region
df_company["Department"] = df_company.groupby("Region")["Department"].transform(lambda x: x.fillna(x.mode()[0]))

In [19]:
# fill in the salary by the average of region and department
df_company["Salary"] = df_company.groupby(["Region", "Department"])["Salary"].transform(lambda x: x.fillna(x.mean()))

Minden régióbol le kell építeni 50%-ot<br>
Vezess be egy új "To_be_fired" oszlopot, ami azoknál az embereknél legyen igaz akik a régióban átlagon alul teljesítenek

In [30]:
df_company.drop(columns=["To_be_fired"], inplace=True)
df_company.groupby("Region")["Performance_Score"].mean()

Region
Budapest     5.438776
Debrecen     5.594059
Kistarcsa    5.285714
Miskolc      5.540541
Szeged       5.207547
Name: Performance_Score, dtype: float64

In [31]:
df_company["To_be_fired"] = df_company.groupby("Region")["Performance_Score"].transform(lambda x: x < x.mean())

In [32]:
df_company.head()

,Date,Region,Department,Salary,Performance_Score,To_be_fired
0,2023-01-01,Miskolc,HR,45249.000000,8,False
1,2023-01-02,Debrecen,Finance,59201.000000,4,True
2,2023-01-03,Budapest,HR,98576.000000,8,False
3,2023-01-04,Miskolc,Sales,84445.956522,7,False
4,2023-01-05,Szeged,Finance,83129.538462,5,True


A leépítés után majd dobni kell egy csontot a bentmaradóknak, szóval hozz létre egy promotion_sum nevű oszlopot, amiben tárold el minden nem kirúgandó ember fizetése és a régió adott pozíciójának átlagfizetése közti különbséget. 

In [34]:
df_company.groupby(["Region", "Department"])["Salary"].mean()

Region     Department
Budapest   Finance       74839.333333
           HR            78428.409091
           IT            81328.764706
           Marketing     77478.000000
           Sales         76610.285714
Debrecen   Finance       68670.086957
           HR            78334.769231
           IT            85543.950000
           Marketing     81047.315789
           Sales         93575.318182
Kistarcsa  Finance       84925.600000
           HR            77613.583333
           IT            74478.562500
           Marketing     82060.000000
           Sales         72942.812500
Miskolc    Finance       74442.950000
           HR            82553.400000
           IT            77511.214286
           Marketing     78532.260870
           Sales         84445.956522
Szeged     Finance       83129.538462
           HR            73648.842105
           IT            87001.454545
           Marketing     76703.772727
           Sales         82839.785714
Name: Salary, dtype: float64

In [50]:
df_company.drop(columns=["Promotion_sum"], inplace=True)

In [51]:
# those who are not to be fired shall earn the same as the average per profession in their region
df_company.loc[df_company["To_be_fired"] == False, "Promotion_sum"] = df_company.groupby(["Region", "Department"])["Salary"].transform(lambda x: x.mean())

In [52]:
df_company["Promotion_sum"] = df_company.groupby(["Region", "Department"])["Salary"].transform(lambda x: x.mean())

In [53]:
df_company.loc[df_company["To_be_fired"] == False, "Promotion_sum"] = df_company.loc[df_company["To_be_fired"] == False, "Promotion_sum"] - df_company.loc[df_company["To_be_fired"] == False, "Salary"]

In [54]:
df_company.loc[df_company["To_be_fired"] == True, "Promotion_sum"] = 0

In [55]:
df_company

,Date,Region,Department,Salary,Performance_Score,To_be_fired,Promotion_sum
0,2023-01-01,Miskolc,HR,45249.000000,8,False,37304.400000
1,2023-01-02,Debrecen,Finance,59201.000000,4,True,0.000000
2,2023-01-03,Budapest,HR,98576.000000,8,False,-20147.590909
3,2023-01-04,Miskolc,Sales,84445.956522,7,False,0.000000
4,2023-01-05,Szeged,Finance,83129.538462,5,True,0.000000
...,...,...,...,...,...,...,...
495,2024-05-10,Miskolc,IT,55892.000000,6,False,21619.214286
496,2024-05-11,Budapest,Finance,87299.000000,1,True,0.000000
497,2024-05-12,Szeged,Sales,57073.000000,4,True,0.000000
498,2024-05-13,Miskolc,Sales,70682.000000,1,True,0.000000
